In [32]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import Row
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.tree import GradientBoostedTrees

In [3]:
ctx = SparkSession.builder \
    .master("spark://hs:7077") \
    .appName("bosch spark") \
    .getOrCreate()
file_dir = "/home/h/kaggle_fun/bosch/raw_data/"

** - 1 change dataframe to vector ** 
      [labeled point](http://stackoverflow.com/questions/32556178/create-labeledpoints-from-spark-dataframe-in-python)
      [比较全的doc](http://www.techpoweredmath.com/spark-dataframes-mllib-tutorial/)    
** - 2 missing values count **   
** - 3 fill missing values **   
** - 4 plot feature point distribution **  
** - 5 normalize the feature **  
** - 6 adjust positive negative ratio **  
** - 7 train model **    
** - 8 evaluate model matric **   
** - 9 [feature engineer](http://stackoverflow.com/questions/32982425/encode-and-assemble-multiple-features-in-pyspark) **   
** -10 [spark python vs scala](http://stackoverflow.com/questions/32464122/spark-performance-for-scala-vs-python) **  
sadly finished already!



In [28]:
def load_csv_df(file_name):
    return ctx.read.csv(file_dir + file_name, header=True, inferSchema=True)

def check_p_n_cnt(input_df, col_name, pos):
    tmp_table_name = "check_p_n_input_df"
    input_df.registerTempTable(tmp_table_name)
    total = ctx.sql("select count(*) as tcnt from %s" % tmp_table_name)
    pos = ctx.sql("select count(*) as pcnt from %s where %s = %s" %(tmp_table_name, col_name, pos))
    print "positive/total : %s/%s" % (pos.collect()[0]["pcnt"], total.collect()[0]["tcnt"])

def adjust_p_n_ratio(input_df, cate_col_name, scale, raw_ratio):
    tmp_table_name = "adjust_p_n_input_df"
    input_df.registerTempTable(tmp_table_name)
    adjusted_df = ctx.sql("""select * from %s where %s = 1 or (%s = 0 and rand() < %s*%s)"""
                          % (tmp_table_name, cate_col_name, cate_col_name, scale, raw_ratio))
    return adjusted_df

def df_to_vec(input_df, ignore_list=[]):
    assembler = VectorAssembler(
        inputCols=[x for x in input_df.columns if x not in ignore_list],
        outputCol='features')
    return assembler.transform(input_df)

def df_to_vec_2(input_df, ignore_list = []):
    labeled_points = input_df.rdd.map(lambda line:LabeledPoint(int(line[-1]),line[0:-1]))
    return labeled_points

def get_corr_matrix(vec, method="pearson"):
    return vec.stat.corr(vec)

def train_lr_model(input_vec):
    lrm = LogisticRegressionWithSGD.train(input_vec, iterations=10)
    return lrm

def train_gbdt_model(input_vec):
    gbdt_model = GradientBoostedTrees.trainClassifier(input_vec,{}, numIterations = 100)
    return gbdt_model

In [5]:
train_numer_df = load_csv_df("train_numeric.csv")
test_numer_df = load_csv_df("test_numeric.csv")

In [22]:
#check_p_n_cnt(train_numer_df, "Response", "1") #6879/1183747
adjusted_train_num = adjust_p_n_ratio(train_numer_df, "Response", 1.2, float(6879)/1183747)
print "pos", adjusted_train_num.filter(adjusted_train_num.Response == 1).count()
print "neg", adjusted_train_num.filter(adjusted_train_num.Response == 0).count()

pos 6879
neg 8200


In [9]:
#train the model
train_numer_vec = df_to_vec_2(adjusted_train_num)
gbdt_model = train_gbdt_model(train_numer_vec)

In [11]:
test_numer_vec = df_to_vec_2(test_numer_df)
predict_res = gbdt_model.predict(test_numer_vec)

In [23]:
print test_numer_vec.take(3)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 124.0 failed 1 times, most recent failure: Lost task 0.0 in stage 124.0 (TID 1809, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/h/soft/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/home/h/soft/spark/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/h/soft/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/h/soft/spark/python/pyspark/rdd.py", line 1306, in takeUpToNumLeft
    yield next(iterator)
  File "<ipython-input-5-e21e388a1c5e>", line 25, in <lambda>
TypeError: int() argument must be a string or a number, not 'NoneType'

In [12]:
predict_res.saveAsTextFile("predict_result.csv")

Py4JJavaError: An error occurred while calling o117.saveAsTextFile.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 117.0 failed 1 times, most recent failure: Lost task 2.0 in stage 117.0 (TID 1751, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/h/soft/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/home/h/soft/spark/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/h/soft/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-5-e21e388a1c5e>", line 25, in <lambda>
TypeError: int() argument must be a string or a number, not 'NoneType'

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.apache.spark.api.python.PythonRunner$$anon$1

In [7]:
train_numer_vec = df_to_vec_2(train_numer_df)
print train_numer_vec.take(3)
#first_3 = train_numer_df.take(3)
#for line in first_3:
#    LabeledPoint(line[-1],line[0:-1])

[LabeledPoint(0.0, [4.0,0.03,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.02,0.083,-0.273,-0.273,-0.271,0.167,-0.213,-0.023,-0.192,-0.088,0.001,0.0,0.01,-0.223,-0.03,nan,nan,nan,nan,nan,nan,nan,nan,nan,-0.001,-0.004,nan,nan,nan,nan,nan,-0.164,-0.077,0.06,-0.157,0.0,0.001,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.018,-0.136,-0.013,0.106,-0.112,-0.131,-0.021,-0.097,0.26,0.012,-0.115,0.113,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na

In [19]:
#train_numer_vec = df_to_vec(train_numer_df, ["Id"])
#train_numer_vec.corr(method="pearson")
ignore_list = ["Response"]
assembler = VectorAssembler(
        inputCols=[x for x in train_numer_df.columns if x not in ignore_list],
        outputCol='features')
#print type(train_numer_vec)
train_numer_vec = assembler.transform(train_numer_df)